In [1]:
import os
import pandas as pd
import natsort
data_dir =  'korea_data'
data_per = os.listdir(data_dir)

In [2]:
def max_frame(data_per):
    max_frames = 0
    for person in data_per:
        words_dir= f'{data_dir}/{person}'
        Id = os.listdir(words_dir)
        for id in Id:
            id_dir = f'{data_dir}/{person}/{id}'
            num = os.listdir(id_dir)
            for n  in num:
                #print("n is", n)
                words_dir_2 = f'{data_dir}/{person}/{id}/{n}/crop_lip'
                frames = os.listdir(words_dir_2)
                if frames == '.DS_Store':
                    continue
                max_frames = max(len(frames), max_frames)
    return max_frames

In [4]:
def concat_x_data(x_data, x_data_):
    if x_data_.shape[0] == max_frame(data_per):
        return np.concatenate([x_data, x_data_], axis = 1) 
    else: 
        pad = max_frame(data_per) - x_data_.shape[1]
        print("pad for ",pad)
        zero_pad = np.zeros((1, pad, 64, 64, 3))
        print(zero_pad.shape)
#         print(x_data_.shape)
        data = np.concatenate([x_data_, zero_pad], axis = 1) #padding
        return data

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class DataAugmentation:
    @classmethod
    def generate(self, img):
        img_array = image.array_to_img(img)
        samples = np.expand_dims(img_array, axis=0)
        
        # generatior 생성
        datagen = ImageDataGenerator(
            width_shift_range=[-10,10], 
            height_shift_range=0.5, 
            rotation_range=45)
        
        it = datagen.flow(samples, batch_size=1)
        augmented = it.next()
        img_augmented = augmented[0]
        img_augmented /= 255. 
        img_augmented = np.expand_dims(img_augmented, 0)

        return img_augmented

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

for person in natsort.natsorted(data_per):
    if person == '.DS_Store':
        continue
    #phrases_dir = f'{data_dir}/{person}/phrases'
    words_dir = f'{data_dir}/{person}'
    Id = os.listdir(words_dir)
    for sentence in natsort.natsorted(Id) :
        if sentence == '.DS_Store':
            continue
        id_dir = f'{data_dir}/{person}/{sentence}'
        num = os.listdir(id_dir)
        for n  in natsort.natsorted(num):
            if n == '.DS_Store':
                continue
            print("person: {}, sentence: {}, n: {} ".format(person, sentence, n))
            words_dir_2 = f'{data_dir}/{person}/{sentence}/{n}/crop_lip'
            frames = os.listdir(words_dir_2)
            x_data_ = np.empty(4)
            x_data_aug = np.empty(4)
            for frame in natsort.natsorted(frames):
                if frame == '.DS_Store':
                    continue
                img_path = f'{words_dir_2}/{frame}'
                
                # flatten 제거한 frame processing
                img = image.load_img(img_path, target_size=(64, 64))
                img_tensor = image.img_to_array(img) #tensor로 type 변경
                img_tensor /= 255. # scaling 
                img_tensor = np.expand_dims(img_tensor, axis = 0) # (1, 64, 64, 3) 차원
                count = 0
                # frame 별 묶기
                if count == 0:
                    x_data_ = img_tensor
                    x_data_aug = DataAugmentation().generate(img)
                    count += 1
                else:
                    x_data_ = np.concatenate([x_data_, img_tensor], axis = 0)
                    img_augmetned = DataAugmentation.generate(img)
                    x_data_aug = np.concatenate([x_data_aug, img_augmetned], axis=0)
                
                print(x_data_.shape)
                
            # 이미지 전체 통합시키기
            x_data_ = np.expand_dims(x_data_, axis = 0)  #[1, frame, 64, 64, 3]
            x_data_aug = np.expand_dims(x_data_aug, axis=0)
            if (n == '00' and sentence == '00' and person == 'A'):
                x_data = x_data_
                x_data = concat_x_data(x_data, x_data_)
                y_data = np.array([int(sentence)])
                data_aug = concat_x_data(x_data, x_data_aug)
                x_data = np.concatenate([x_data, data_aug], axis = 0)
                y_data = np.concatenate([y_data, [int(sentence)]])
            else:
                if x_data_.shape[0] == max_frame(data_per):
                    x_data = concat_x_data(x_data, x_data_)
                    y_data = np.concatenate([y_data, [int(sentence)]])
                    x_data = concat_x_data(x_data, x_data_aug)
                    y_data = np.concatenate([y_data, [int(sentence)]])
                else: 
                    data = concat_x_data(x_data, x_data_)
                    x_data = np.concatenate([x_data, data], axis = 0)
                    y_data = np.concatenate([y_data, [int(sentence)]])
                    data_aug = concat_x_data(x_data, x_data_aug)
                    x_data = np.concatenate([x_data, data_aug], axis = 0)
                    y_data = np.concatenate([y_data, [int(sentence)]])
            print(x_data.shape)
            print("y.shape: {}" .format(y_data.shape))

person: A, sentence: 00, n: 00 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(2, 82, 64, 64, 3)
y.shape: (2,)
person: A, sentence: 00, n: 01 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(4, 82, 64, 64, 3)
y.shape: (4,)
person: A, sentence: 00, n: 02 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 6

pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(34, 82, 64, 64, 3)
y.shape: (34,)
person: A, sentence: 05, n: 02 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(36, 82, 64, 64, 3)
y.shape: (36,)
person: A, sentence: 06, n: 00 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(38, 82, 64, 64, 3)
y.shape: (38,)
person: A, sentence: 06, n: 01 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3

pad for  81
(1, 81, 64, 64, 3)
(72, 82, 64, 64, 3)
y.shape: (72,)
person: A, sentence: 12, n: 00 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(74, 82, 64, 64, 3)
y.shape: (74,)
person: A, sentence: 12, n: 01 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 

pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(112, 82, 64, 64, 3)
y.shape: (112,)
person: B, sentence: 02, n: 02 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(114, 82, 64, 64, 3)
y.shape: (114,)
person: B, sentence: 03, n: 00 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81

pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(150, 82, 64, 64, 3)
y.shape: (150,)
person: B, sentence: 09, n: 00 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(152, 82, 64, 64, 3)
y.shape: (152,)
person: B, sentence: 09, n: 01 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(154, 82, 64, 64, 3)
y.shape: (154,)
person: B, sentence: 09, n: 02 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(156, 82, 64, 64, 3)
y.shape: (156,)
person: B, sentence: 10, n: 00 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64,

...

(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(884, 82, 64, 64, 3)
y.shape: (884,)
person: J, sentence: 03, n: 01 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(886, 82, 64, 64, 3)
y.shape: (886,)
person: J, sentence: 03, n: 02 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(888, 82, 64, 64, 3)
y.shape: (888,)
person: J, sentence: 04, n: 00 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64,

(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(938, 82, 64, 64, 3)
y.shape: (938,)
person: J, sentence: 12, n: 01 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(940, 82, 64, 64, 3)
y.shape: (940,)
person: J, sentence: 12, n: 02 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(942, 82, 64, 64, 3)
y.shape: (942,)
person: J, sentence: 13, n: 00 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64

pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(990, 82, 64, 64, 3)
y.shape: (990,)
person: K, sentence: 05, n: 00 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(992, 82, 64, 64, 3)
y.shape: (992,)
person: K, sentence: 05, n: 01 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(994, 82, 64, 64, 3)
y.shape: (994,)
person: K, sentence: 05, n: 02 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(996, 82, 64, 64, 3)
y.shape: (996,)
person: K, se

pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(1042, 82, 64, 64, 3)
y.shape: (1042,)
person: K, sentence: 13, n: 02 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(1044, 82, 64, 64, 3)
y.shape: (1044,)
person: K, sentence: 14, n: 00 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(1046, 82, 64, 64, 3)
y.shape: (1046,)
person: K, sentence: 14, n: 01 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(1048, 82, 64, 64, 3)
y.shape: (1048,)
person: K, sentence: 14, n: 02 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1,

pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(1094, 82, 64, 64, 3)
y.shape: (1094,)
person: L, sentence: 06, n: 01 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(1096, 82, 64, 64, 3)
y.shape: (1096,)
person: L, sentence: 06, n: 02 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
pad for  81
(1, 81, 64, 64, 3)
(1098, 82, 64, 64, 3)
y.shape: (1098,)
person: L, sentence: 07, n: 00 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(

In [ ]:
with open('D:/git/LipService/face_x_data.pickle', 'wb') as f:
    pickle.dump(x_data, f)
with open('D:/git/LipService/face_x_data.pickle', 'wb') as f:
    pickle.dump(y_data, f)

In [8]:
np.save('D:/git/LipService/face_x_data', x_data) # x_save.npy
np.save('D:/git/LipService/face_y_data', y_data) # y_data.npy

In [ ]:
# # load
# with open('./text2keypoint/tacotron/data/y_data.pickle', 'rb') as f:
#     y = pickle.load(f)